##Install

In [5]:
!pip install rectools==0.3.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 6.6 MB/s 
     |████████████████████████████████| 13.5 MB 45.5 MB/s 
     |████████████████████████████████| 1.1 MB 52.1 MB/s 
     |████████████████████████████████| 88 kB 8.3 MB/s 
     |████████████████████████████████| 60 kB 7.5 MB/s 
     |████████████████████████████████| 310 kB 48.0 MB/s 
     |████████████████████████████████| 188 kB 50.8 MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3411055 sha256=72cc522649c0e51a6e9647057c990f8a41040bf5b08530ab47ca35fd0f9340cf
  Stored in directory: /root/.cache/pip/wheels/44/7e/7d/a17324ea207cfbe76aca878b5b8ca0aa932cf55d163329be37
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705355 sha256=c5b1e2da2dd48eca0f3d565e03645f5f794e6063219984f70fcc6b3d7b0616e3
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3b

# Метрика Pfound

## Imports

In [6]:
import pandas as pd
import numpy as np
import numba as nb

from tqdm.auto import tqdm
from rectools import Columns

In [7]:
np.random.seed(23)

## Homework

### PFound
Исходные данные - Yandex Cup 2022 Analytics
- Ссылка - https://yandex.ru/cup/analytics/analysis/ , пример A. Рассчитать pFound
- Данные - https://yadi.sk/d/guqki4UI4hFlXQ
- Формула
$$pFound@K = \sum_{i=1}^{k} pLook[i]\ pRel[i]$$

$$pLook[1] = 1$$

$$pLook[i] = pLook[i-1]\ (1 - pRel[i-1])\ (1 - pBreak)$$

$$pBreak = 0.15$$

**Задача** - написать функцию, которая принимает на вход dataframe (после join), а на выходе дает средний pFound по всем query.
- Запрещается использовать циклы for для расчет метрики (как полностью, так и ее частей).
- Усложнение, если задача показалась легкой - попробуйте обойтись без groupby (не уверен, что это возможно, но вдруг вы справитесь)


##Решение с циклом

(Измененный пример из yandexcup)


In [10]:
	# считываем данные
qid_query = pd.read_csv("qid_query.tsv", sep="\t", names=["qid", "query"])
qid_url_rating = pd.read_csv("qid_url_rating.tsv", sep="\t", names=["qid", "url", "rating"])
hostid_url = pd.read_csv("hostid_url.tsv", sep="\t", names=["hostid", "url"])

	# делаем join двух таблиц, чтобы было просто брать url с максимальным рейтингом
qid_url_rating_hostid = pd.merge(qid_url_rating, hostid_url, on="url")
qid_url_rating_hostid.head()

,qid,url,rating,hostid
0,10387,http://batman-arkhamcity.ru/,0.00,64
1,10387,http://bigtorrents.org/publ/batman_arkham_city...,0.14,71
2,10387,http://consolelife.ru/xbox-360/6577-o-rossiysk...,0.14,101
3,10387,http://dic.academic.ru/book.nsf/3662736/Batman...,0.00,115
4,10387,http://forum.csmania.ru/viewtopic.php?t=25986,0.14,155


In [27]:
def plook(ind, rels):
  if ind == 0:
    return 1
  return plook(ind-1, rels)*(1-rels[ind-1])*(1-0.15)


def pfound(group, top_k=2):
  max_by_host = group.groupby("hostid")["rating"].max() # максимальный рейтинг хоста
  topk = max_by_host.sort_values(ascending=False)[:top_k] # берем топ k урлов с наивысшим рейтингом
  pfound = 0
  for ind, val in enumerate(topk):
    pfound += val*plook(ind, topk.values)
  return pfound

In [28]:
qid_pfound = qid_url_rating_hostid.groupby('qid').apply(pfound) # группируем по qid и вычисляем pfound
qid_pfound.head()

qid
10387     0.24234
20860     0.48021
21070     0.24234
35618     0.24234
107538    0.19117
dtype: float64

In [35]:
qid_pfound = qid_url_rating_hostid.groupby('qid').apply(pfound, top_k=10).mean() # группируем по qid и вычисляем средний pfound по всем query
qid_pfound

0.4603173929969002

## Решение без цикла


In [47]:
def headtail(df):
    return pd.concat([df.head(), df.tail()])

In [52]:
	# считываем данные
qid_query = pd.read_csv("qid_query.tsv", sep="\t", names=["qid", "query"])
qid_url_rating = pd.read_csv("qid_url_rating.tsv", sep="\t", names=["qid", "url", "rating"])
hostid_url = pd.read_csv("hostid_url.tsv", sep="\t", names=["hostid", "url"])

	# делаем join двух таблиц, чтобы было просто брать url с максимальным рейтингом
qid_url_rating_hostid = pd.merge(qid_url_rating, hostid_url, on="url")

#Удаляем ненужные колонки
qid_url_rating_hostid = qid_url_rating_hostid.drop(["url"], axis=1)
print(headtail(qid_url_rating_hostid))
qid_url_rating_hostid.sort_values(by=['rating'], ascending=False, inplace=True)
headtail(qid_url_rating_hostid)

        qid  rating  hostid
0     10387    0.00      64
1     10387    0.14      71
2     10387    0.14     101
3     10387    0.00     115
4     10387    0.14     155
849  380923    0.00    1063
850  380923    0.00    1065
851  380923    0.00    1105
852  380923    0.00    1168
853  380923    0.07    1172


,qid,rating,hostid
567,295761,0.61,565
566,295761,0.61,564
565,295761,0.61,564
652,347852,0.41,590
575,295761,0.41,824
485,221830,0.00,659
189,168170,0.00,658
190,168170,0.00,667
193,168170,0.00,752
0,10387,0.00,64


In [55]:
qid_url_rating_hostid["rank"] = qid_url_rating_hostid.groupby("qid").cumcount() + 1
headtail(qid_url_rating_hostid)

,qid,rating,hostid,rank
567,295761,0.61,565,1
566,295761,0.61,564,2
565,295761,0.61,564,3
652,347852,0.41,590,1
575,295761,0.41,824,4
485,221830,0.00,659,51
189,168170,0.00,658,49
190,168170,0.00,667,50
193,168170,0.00,752,51
0,10387,0.00,64,52


In [51]:
qid_url_rating_hostid["1rel"] = qid_url_rating_hostid.groupby("qid")["rating"].cumsum()
headtail(qid_url_rating_hostid)

,qid,rating,hostid,cumR
567,295761,0.61,565,0.61
566,295761,0.61,564,1.22
565,295761,0.61,564,1.83
652,347852,0.41,590,0.41
575,295761,0.41,824,2.24
485,221830,0.00,659,4.34
189,168170,0.00,658,1.82
190,168170,0.00,667,1.82
193,168170,0.00,752,1.82
0,10387,0.00,64,2.73
